In [1]:
import torch, logging
logging.disable(logging.WARNING)  
from transformers import CLIPTextModel, CLIPTokenizer

In [2]:
# TODO use stable diffusion safetensor file
# openai/clip-vit-large-patch14
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16)
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16).to("cuda")

In [5]:
prompt = ["a dog wearing hat"]
tok = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt") 
print(f'Words shape: {tok.input_ids.shape}')
print(f'Attention shape: {tok.attention_mask.shape}')
tok

Words shape: torch.Size([1, 77])
Attention shape: torch.Size([1, 77])


{'input_ids': tensor([[49406,   320,  1929,  3309,  3801, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]])}

In [18]:
for token in tok.input_ids[0,:6]:
    print(f"{token}:{tokenizer.convert_ids_to_tokens(token.item())}")

49406:<|startoftext|>
320:a</w>
1929:dog</w>
3309:wearing</w>
3801:hat</w>
49407:<|endoftext|>


In [19]:
emb = text_encoder(tok.input_ids.to("cuda"))[0].half()
print(f"Shape of embedding : {emb.shape}")
emb

Shape of embedding : torch.Size([1, 77, 768])


tensor([[[-0.3884,  0.0229, -0.0522,  ..., -0.4902, -0.3066,  0.0674],
         [ 0.0278, -1.3262,  0.3086,  ..., -0.5239,  0.9731,  0.6660],
         [-1.5928,  0.5059,  1.0801,  ..., -1.5273, -0.8433,  0.1587],
         ...,
         [-1.4707,  0.3135,  1.1670,  ...,  0.3743,  0.5376, -1.5039],
         [-1.4697,  0.3022,  1.1777,  ...,  0.3752,  0.5420, -1.5000],
         [-1.4424,  0.3142,  1.1982,  ...,  0.3516,  0.5420, -1.5498]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<NativeLayerNormBackward0>)